In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import halo_data as hd


In [ ]:
data = hd.getdata("C:/Users/LV/OneDrive - University of Helsinki/FMI/halo/53/depolarization/")
# data = hd.getdata("G:/OneDrive - University of Helsinki/FMI/halo/53/depolarization/")
# data = hd.getdata(r'G:\OneDrive - University of Helsinki\FMI\halo\53\depolarization')


In [ ]:
file_name = next(data)
df = hd.halo_data(file_name)
df.info
df.full_data
df.full_data_names
df.data
# Names of data
df.data_names
# More info
df.more_info
# Get meta data of each variable
df.meta_data('co_signal')

# Get meta data of all variables
{'==>' + key: df.meta_data(key) for key in df.full_data_names}

# Only crucial info
{'==>' + key: df.meta_data(key)['_attributes'] for key in df.full_data_names}


In [ ]:
# Change masking missing values from -999 to NaN
df.unmask999()
# Overview of data
df.describe()


In [ ]:
# Plot data
%matplotlib inline
df.plot(
    variables=['beta_raw', 'v_raw', 'cross_signal', 'depo_raw', 'co_signal',
               'cross_signal_averaged', 'depo_averaged_raw', 'co_signal_averaged'], ncol=2, size=(20, 15))


In [ ]:
%matplotlib qt
fig, ax = plt.subplots(1, 2, figsize=(24, 24))
p = ax[0].pcolormesh(df.data['time'],
                     df.data['range'],
                     df.data['co_signal'].transpose(),
                     cmap='jet', vmin=0.995, vmax=1.005)
area = hd.area_histogram(ax[0], ax[1], fig, df.data['time'],
                         df.data['range'],
                         df.data['co_signal'].transpose(),
                         hist=False)
fig.colorbar(p, ax=ax[0])

In [ ]:
noise = area.area - 1
threshold = 1 + np.nanmean(noise) + np.nanstd(noise) * 2

threshold


In [ ]:
%matplotlib qt
fig, ax = plt.subplots(1, 2, figsize=(24, 24))
p = ax[0].pcolormesh(df.data['time_averaged'],
                     df.data['range'],
                     df.data['co_signal_averaged'].transpose(),
                     cmap='jet', vmin=0.995, vmax=1.005)
area = hd.area_histogram(ax[0], ax[1], fig, df.data['time_averaged'],
                         df.data['range'],
                         df.data['co_signal_averaged'].transpose(),
                         hist=False)
fig.colorbar(p, ax=ax[0])

In [ ]:
noise_averaged = area.area - 1
threshold_averaged = 1 + np.nanmean(noise_averaged) + np.nanstd(noise_averaged) * 2

threshold_averaged



In [ ]:
df.filter(variables=['beta_raw', 'v_raw', 'cross_signal', 'depo_raw'],
          ref='co_signal', threshold=threshold)

df.filter(variables=['cross_signal_averaged', 'depo_averaged_raw'],
          ref='co_signal_averaged', threshold=threshold_averaged)


In [ ]:
# Plot data
%matplotlib inline
df.plot(
    variables=['beta_raw', 'v_raw', 'cross_signal', 'depo_raw', 'co_signal',
               'cross_signal_averaged', 'depo_averaged_raw', 'co_signal_averaged'], ncol=2, size=(20, 15))



In [ ]:
df.describe()